In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf 
from cartopy import crs

hv.extension('matplotlib', 'bokeh')

The GeoViews package provides a library of [HoloViews](https://holoviews.org) Element types which make it very easy to plot data on various geographic projections and other utilities to plot in geographic coordinate systems. Elements are very simple wrappers around the data the only thing that distinguishes a GeoViews element from a HoloViews one is the addition of a ``crs`` parameter, which defines a cartopy coordinate reference system declaring the coordinate system of the data. This allows GeoViews to automatically project the data. By default all elements assume a ``PlateCarree`` projection (also sometimes known as the equirectangular projection), which lets you define the data in longitudes and latitudes.

By default the plot will follow the specified ``crs`` when using matplotlib:

In [ ]:
%%output size=400
features = hv.Overlay([gf.ocean, gf.land, gf.rivers, gf.lakes, gf.borders, gf.coastline])
print(gf.land.crs)
features

## Supported projections

The full list of supported projections is as follows:

In [ ]:
projections = [crs.RotatedPole, crs.Mercator, crs.LambertCylindrical, crs.Geostationary, 
               crs.AzimuthalEquidistant, crs.OSGB, crs.EuroPP, crs.Gnomonic, crs.PlateCarree, 
               crs.Mollweide, crs.OSNI, crs.Miller, crs.InterruptedGoodeHomolosine,
               crs.LambertConformal, crs.SouthPolarStereo, crs.AlbersEqualArea, crs.Orthographic,
               crs.NorthPolarStereo, crs.Robinson]

If we want to display the plot in a different coordinate system than the data is defined in we can declare the ``projection`` as a plot option:

In [ ]:
hv.Layout([gf.coastline.relabel(group=p.__name__).options(projection=p()) for p in projections])

## Projecting with bokeh

Note however that only the matplotlib backend supports plotting the data in different projections. When using the bokeh plotting extension all data will be projected to Web Mercator coordinates since that is the native coordinate system for interactive tile sources.

In [ ]:
%%output backend='bokeh' size=200
features

## Customizing projections

Many coordinate reference systems provide various parameters to customize certain aspects of the projection, e.g. the Geostationary CRS allows overriding the ``central_longitude`` and ``satellite_height`` to change the view of the globe:

In [ ]:
features.options(projection=crs.Geostationary(central_longitude=-80, satellite_height=35785831*2))

## Explicitly projecting

As we have discovered the plotting system will project data to the desired coordinate system automatically. However in certain cases this can be quite expensive especially if it occurs multiple times. Therefore GeoViews makes a high-level operation available to project most types of data from their native ``crs`` to the declared projection. Here we will declare a ``Points`` element containing the longitude/latitude locations of a number of cities. Using the ``gv.operation.project`` operation we can easily project the data to Mercator coordinates:

In [ ]:
cities = gv.Points([(-74.01, 40.71, 'New York'), (0.13, 51.51, 'London'), (116.40, 39.9, 'Beijing')], vdims='City')
projected = gv.operation.project(cities, projection=crs.GOOGLE_MERCATOR)
print(projected.crs)
projected.dframe()